# Image Recognition Project - Structural Defect Recognition
---------------------------------------------------------------
## Data Visualisation

### Section Objectives
 - Complete the project objective and answer the business requirements by training the -

 --------------------------------------------------------------

## Import Libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
sns.set_style("white")
from matplotlib.image import imread

## Setting Working Directory

In [ ]:
cwd= os.getcwd()

In [ ]:
os.chdir('/workspaces/Image_Recognition_Project')
print("You set a new current directory")

In [ ]:
work_dir = os.getcwd()
work_dir

## Setting Input and Output Directories

### Input:

### Output: